In [1]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sionna.phy


import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.phy.config.seed = 42 # Set seed for reproducible results

E0000 00:00:1745432976.398628   50097 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745432976.408854   50097 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745432976.425152   50097 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745432976.425169   50097 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745432976.425173   50097 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745432976.425175   50097 computation_placer.cc:177] computation placer already registered. Please check linka

In [2]:
import sys
sys.path.append('../')
from src import SystemConfig, UeConfig, MyConfig, MySimulator, MyPUSCHConfig

In [3]:
from src.my_pusch_config import MyPUSCHConfig
from src.my_encoder import MyTBEncoder
from src.my_decoder import MyTBDecoder

from sionna.phy.mapping import BinarySource, Mapper
from sionna.phy.nr import PUSCHPilotPattern, LayerMapper, LayerDemapper, PUSCHLSChannelEstimator
from sionna.phy.ofdm import ResourceGrid, ResourceGridMapper, LinearDetector
from sionna.phy.nr.utils import generate_prng_seq
from sionna.phy.channel import AWGN, OFDMChannel, gen_single_sector_topology as gen_topology
from sionna.phy.mimo import StreamManagement
from sionna.phy.channel.tr38901 import Antenna, AntennaArray, UMi, UMa, CDL
from sionna.phy import Block


import tensorflow as tf
import numpy as np

In [4]:
sys_cfg = SystemConfig(
    NCellId=442
)
ue_cfg = UeConfig(NPrb=162)
my_cfg = MyConfig(sys_cfg, ue_cfg)
my_pusch_cfg = MyPUSCHConfig(my_config=my_cfg)

In [5]:
my_pusch_cfg.show()

Carrier Configuration
cyclic_prefix : normal
cyclic_prefix_length : 2.3437500000000002e-06
frame_duration : 0.01
frame_number : 0
kappa : 64.0
mu : 1
n_cell_id : 442
n_size_grid : 162
n_start_grid : 0
num_slots_per_frame : 20
num_slots_per_subframe : 2
num_symbols_per_slot : 14
slot_number : 4
sub_frame_duration : 0.001
subcarrier_spacing : 30
t_c : 5.086263020833334e-10
t_s : 3.2552083333333335e-08

PUSCH Configuration
dmrs_grid : shape (1, 1944, 14)
dmrs_grid_precoded : shape ()
dmrs_mask : shape (1944, 14)
dmrs_symbol_indices : [3, 11]
first_resource_block : 0
first_subcarrier : 0
frequency_hopping : neither
l : [3, 11]
l_0 : 3
l_bar : [3, 11]
l_d : 14
l_prime : [0]
l_ref : 0
mapping_type : A
n : shape (486,)
n_rnti : 20002
n_size_bwp : 162
n_start_bwp : 0
num_antenna_ports : 1
num_coded_bits : 46656
num_layers : 1
num_ov : 0
num_res_per_prb : 144
num_resource_blocks : 162
num_subcarriers : 1944
phy_cell_id : 442
precoding : non-codebook
precoding_matrix : None
symbol_allocation : [

In [6]:
my_simulator = MySimulator(my_pusch_cfg)
my_simulator.reference

<tf.Tensor: shape=(1, 1, 14, 1944), dtype=complex64, numpy=
array([[[[ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
         ...,
         [-1.+1.j,  0.+0.j,  1.-1.j, ...,  0.+0.j, -1.-1.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j]]]],
      dtype=complex64)>

In [7]:
my_simulator.get_r_rg(4)

<tf.Tensor: shape=(4, 1, 1, 14, 1944), dtype=complex64, numpy=
array([[[[[ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
          ...,
          [-1.+1.j,  0.+0.j,  1.-1.j, ...,  0.+0.j, -1.-1.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j]]]],



       [[[[ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
          ...,
          [-1.+1.j,  0.+0.j,  1.-1.j, ...,  0.+0.j, -1.-1.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j]]]],



       [[[[ 0.+0.j,  0.+0.j,  0.+0.j, ..., 

In [8]:
b,c,y,x = my_simulator(2)

AttributeError: in user code:

    File "/workspaces/dsp/notebook/../src/my_simulator.py", line 220, in call  *
        no = tf.math.reduce_variance(y, axis=[-1,-2,-3,-4]) / self._snr

    AttributeError: 'MySimulator' object has no attribute '_snr'


In [13]:
my_simulator.get_c_rg(c)

<tf.Tensor: shape=(2, 1, 1, 14, 1944, 2), dtype=float32, numpy=
array([[[[[[0., 0.],
           [1., 1.],
           [0., 0.],
           ...,
           [0., 0.],
           [1., 1.],
           [0., 0.]],

          [[1., 0.],
           [0., 0.],
           [0., 0.],
           ...,
           [1., 0.],
           [0., 0.],
           [0., 0.]],

          [[1., 0.],
           [1., 0.],
           [1., 1.],
           ...,
           [0., 1.],
           [0., 1.],
           [0., 0.]],

          ...,

          [[0., 0.],
           [0., 0.],
           [0., 0.],
           ...,
           [0., 0.],
           [0., 0.],
           [0., 0.]],

          [[0., 0.],
           [0., 0.],
           [1., 1.],
           ...,
           [0., 0.],
           [1., 1.],
           [1., 0.]],

          [[0., 1.],
           [0., 1.],
           [1., 0.],
           ...,
           [1., 1.],
           [1., 0.],
           [1., 1.]]]]],




       [[[[[0., 1.],
           [0., 1.],
        

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import tensorflow as tf
import os
import tempfile

class MyWriter:
    def __init__(self, num_shards=256,
                 samples_per_shard=1024,
                 max_workers=8,
                 output_dir="tfrecords",
                 isTempfile=False):
        
        self.features_order = ['b', 'c', 'y', 'x', 'r']  # Thứ tự tensor trả về từ _get_example
        self.feature = None
        self.selected_indexes = []

        self.samples_per_shard = samples_per_shard
        self.num_shards = num_shards
        self.max_workers = max_workers

        self.output_dir = output_dir
        if isTempfile:
            self.temp_dir = tempfile.TemporaryDirectory()
            self.output_dir = os.path.join(self.temp_dir.name, output_dir)
        
        os.makedirs(self.output_dir, exist_ok=True)

    def _serialize_example(self, feature_tensors):
        def _bytes_feature(tensor):
            return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(tensor).numpy()]))
        
        # Tạo dictionary feature từ danh sách tên và tensor tương ứng
        feature_dict = {
            name: _bytes_feature(tensor) 
            for name, tensor in zip(self.feature, feature_tensors)
        }
        return tf.train.Example(features=tf.train.Features(feature=feature_dict)).SerializeToString()

    @tf.function(jit_compile=True)
    def _get_example(self, simulator):
        b, c, y, x = simulator(1, 1e-3, return_tx_iq=True)
        r = simulator.get_r_rg(1)
        c = simulator.get_r_rg(c)
        return (b[0, 0], c[0, 0], y[0, 0], x[0, 0], r[0, 0])

    def _write_shard(self, shard_id, progress_callback, simulator):
        shard_path = os.path.join(self.output_dir, f"data-{shard_id:04d}-of-{self.num_shards:04d}.tfrecord")
        buffer = []
        with tf.io.TFRecordWriter(shard_path) as tfwriter:
            for _ in range(self.samples_per_shard):
                # Lấy tất cả các tensor từ simulator
                example_tensors = self._get_example(simulator)
                
                # Lọc các tensor theo danh sách đã chọn
                selected_tensors = [example_tensors[i] for i in self.selected_indexes]
                
                # Serialize và ghi vào buffer
                serialized = self._serialize_example(selected_tensors)
                buffer.append(serialized)

                # Ghi buffer khi đủ 256 mẫu
                if len(buffer) == 256:
                    for example in buffer:
                        tfwriter.write(example)
                        progress_callback()
                    buffer.clear()

            # Ghi phần còn lại trong buffer
            for example in buffer:
                tfwriter.write(example)
                progress_callback()

    def write(self, simulator, feature=['b','c','y','x','r']):
        # Validate feature names
        for f in feature:
            if f not in self.features_order:
                raise ValueError(f"Invalid feature '{f}'. Valid features are: {self.features_order}")
        
        self.feature = feature
        self.selected_indexes = [self.features_order.index(f) for f in self.feature]
        
        simulator = simulator.clone()
        _ = simulator(1, return_tx_iq=True)  # Khởi tạo simulator
        
        total_samples = self.num_shards * self.samples_per_shard
        pbar = tqdm(total=total_samples, desc="Generating TFRecords", unit=" sample")

        def progress_callback_gen():
            return lambda: pbar.update(1)
        
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            futures = []
            for shard_id in range(self.num_shards):
                callback = progress_callback_gen()
                futures.append(executor.submit(
                    self._write_shard, 
                    shard_id, 
                    callback, 
                    simulator.clone()  # Đảm bảo mỗi luồng dùng simulator riêng
                ))

            for future in as_completed(futures):
                future.result()

        pbar.close()

In [15]:
writer = MyWriter(simulator=my_simulator, num_shards=4, samples_per_shard=32, max_workers=4, output_dir="tfrecords")

tfrecords


In [ ]:
writer.write()

Generating TFRecords: 100%|██████████| 600/600 [00:38<00:00, 15.78 sample/s] 


In [ ]:
def parse_tfrecord_fn(example_proto):
    feature_description = {
        "b": tf.io.FixedLenFeature([], tf.string),
        "c": tf.io.FixedLenFeature([], tf.string),
        "y": tf.io.FixedLenFeature([], tf.string),
        "x": tf.io.FixedLenFeature([], tf.string),
    }
    
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    parsed["b"] = tf.io.parse_tensor(parsed["b"], out_type=tf.float32)
    parsed["c"] = tf.io.parse_tensor(parsed["c"], out_type=tf.float32)
    parsed["y"] = tf.io.parse_tensor(parsed["y"], out_type=tf.complex64)
    parsed["x"] = tf.io.parse_tensor(parsed["x"], out_type=tf.complex64)
    return parsed

def load_dataset_from_shards(shard_dir, batch_size=64, shuffle_buffer_size=10000):
    """
    Load and parse all .tfrecord files in `shard_dir` as a batched, prefetched tf.data.Dataset
    """
    file_pattern = os.path.join(shard_dir, "*.tfrecord")
    dataset = tf.data.Dataset.list_files(file_pattern, shuffle=True)

    dataset = dataset.interleave(
        lambda filename: tf.data.TFRecordDataset(filename),
        cycle_length=tf.data.AUTOTUNE,
        num_parallel_calls=tf.data.AUTOTUNE
    )

    dataset = dataset.map(parse_tfrecord_fn, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    dataset = dataset.cache()
    return dataset

In [ ]:
# Load lại
ds = load_dataset_from_shards('tfrecords', batch_size=64)

# Duyệt thử
for batch in ds:
    print(batch["b"].shape, batch["y"].shape)

(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (6

In [ ]:
for batch in ds:
    print(batch["b"].shape, batch["y"].shape)

(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (64, 4, 14, 48)
(64, 128) (6